In [ ]:
!git clone https://github.com/Jacobiano/morpholayers.git

In [ ]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt

plt.gray()
plt.rc("font", family="serif")
plt.rc("xtick", labelsize="x-small")
plt.rc("ytick", labelsize="x-small")

import skimage

print("TensorFlow version:", tensorflow.__version__)
print("Numpy version:", np.__version__)
print("Skimage version:", skimage.__version__)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    MaxPooling2D,
    Flatten,
    Dropout,
    Dense,
)
from matplotlib import pyplot as plt
import numpy as np
from morpholayers.layers import Gradient2D
from morpholayers.constraints import SEconstraint
from morpholayers.initializers import SparseZeros
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape)

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
use_samples = 5000  # Number of samples used in the training

# the data, split between train and test sets
x_train = x_train[0:use_samples]
y_train = y_train[0:use_samples]

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define the text labels
fashion_mnist_labels = [
    "T-shirt/top",  # index 0
    "Trouser",  # index 1
    "Pullover",  # index 2
    "Dress",  # index 3
    "Coat",  # index 4
    "Sandal",  # index 5
    "Shirt",  # index 6
    "Sneaker",  # index 7
    "Bag",  # index 8
    "Ankle boot",
]  # index 9

plt.figure(figsize=(16, 16))
for i in range(25):
    # Image index, you can pick any number between 0 and 59,999
    img_index = i + 1
    # y_train contains the lables, ranging from 0 to 9
    label_index = np.argmax(y_train, axis=1)[img_index]
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[img_index, :, :, 0])
    plt.title((fashion_mnist_labels[label_index]))

In [ ]:
import skimage.morphology as skm

# cross=skm.disk(1)
ownSE = skm.disk(2)
print(ownSE)

In [ ]:
xin = Input(shape=input_shape)
# xlayer=Dilation2D(9, padding='valid',kernel_size=(ownSE.shape[0],ownSE.shape[1]), kernel_initializer='zeros',kernel_constraint = SEconstraint(SE=ownSE))(xin)
# xlayer=Dilation2D(9, padding='valid',kernel_size=(ownSE.shape[0],ownSE.shape[1]), kernel_initializer=RandomLattice(),kernel_constraint = SEconstraint(SE=ownSE))(xin)
# xlayer=Dilation2D(9, padding='valid',kernel_size=(ownSE.shape[0],ownSE.shape[1]), kernel_initializer=SparseZeros(th=.75),kernel_constraint = SEconstraint(SE=ownSE))(xin)
# xlayer=Gradient2D(9, padding='valid',kernel_size=(ownSE.shape[0],ownSE.shape[1]), kernel_initializer=RandomLattice(),kernel_constraint = SEconstraint(SE=ownSE))(xin)
xlayer = Gradient2D(
    9,
    padding="valid",
    kernel_size=(ownSE.shape[0], ownSE.shape[1]),
    kernel_initializer=SparseZeros(th=0.75),
    kernel_constraint=SEconstraint(SE=ownSE),
)(xin)
x = MaxPooling2D(pool_size=(2, 2))(xlayer)
x = Flatten()(x)
x = Dropout(0.5)(x)
xoutput = Dense(num_classes, activation="softmax")(x)
modeli = Model(xin, outputs=xoutput)

In [ ]:
modeli.summary()

**Some Examples of Morphological Layers:**

*   Dilation2D
*   Erosion2D

*   Opening2D
*   Closing2D

*   TopHatClosing2D
*   TopHatOpening2D

*   Gradient2D
*   Probing2D


*   Reconstruction (Check documentation)
*   ToggleMapping (Check documentation)


**Initialization**

* Zeros: Flat SE (Used as default value):

* RandomNormal(mean=0.0, stddev=0.05):

* RandomUniform(minval=-0.05, maxval=0.05)

* Ones: All value on 1. (Non-sense in the case of a morphological layer)

* Quadratic: Quadratic structuring element

* RandomLattice: Random values between (-∞ and ∞)

* RandomNegativeLattice: Random values between (-∞ and 0)

* MinusOnesZeroCenter: Identity SE (0 in the center and -∞ otherwise)

* SignedOnes: Random (-∞ or ∞) values

* SparseZeros: SparseZeros(th=.95),(th*100)% values in -∞ and the rest to 0.

* SparseNumZeros: SparseNumZeros(th=0),(th+1) values in 0 and the rest to -∞.

* RandomwithMinLattice: Random values between (-∞ and ∞) and center to −∞

* RandomwithZeroLattice: Random values between (-∞ and ∞) and center to 0

* RandomwithMaxLattice: Random values between (-∞ and ∞) and center to ∞

In [ ]:
# tensorflow.keras.utils.plot_model(modeli, to_file='model.png',show_shapes=True,show_layer_names=False)
# plt.figure(figsize=(20,15))
# plt.imshow(plt.imread('model.png'))
# plt.axis('off')
# plt.title('Architecture')
# plt.show()
# print('Number of Parameters by Layer')
modeli.summary()

In [ ]:
modeli.compile(
    loss="categorical_crossentropy",
    optimizer=tensorflow.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [ ]:
print("Initialization")
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(modeli.layers[1].weights[0][:, :, 0, i], clim=[-1, 1])
plt.show()

In [ ]:
history = modeli.fit(
    x_train, y_train, batch_size=24, epochs=10, validation_data=(x_test, y_test)
)

In [ ]:
plt.plot(history.history["loss"], label="training loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.grid()
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
plt.plot(history.history["accuracy"], label="Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.grid()
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
modeli.layers

In [ ]:
modeli.layers[1].weights[0].shape

In [ ]:
print("Obtained Structuring Elements")
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(modeli.layers[1].weights[0][:, :, 0, i], clim=[-1, 0])
plt.show()

In [ ]:
modelinterpretation = Model(xin, outputs=xlayer)

In [ ]:
y_act = modelinterpretation.predict(x_train[0:5, :, :, :])

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(y_act[1, :, :, i])
plt.show()

In [ ]:
Y_test = np.argmax(y_test, axis=1)  # Convert one-hot to index
y_pred = np.argmax(modeli.predict(x_test), axis=1)
CM = confusion_matrix(Y_test, y_pred)
print(CM)
plt.imshow(CM, cmap="hot", vmin=0, vmax=1000)
plt.title("Confusion Matrix")
plt.show()
print(classification_report(Y_test, y_pred))